In [447]:
import os
import re
import csv

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.markers import MarkerStyle
import numpy as np
import math


In [448]:
sm = {}
sm["directory"]= "20230727"
sm["file_includes"] = "_sm_"


p = re.compile(r"\d\d\d")
instance_size_regex = re.compile(r"\d\d\d\d")

GROUNDING_TIME = "grounding_time"
GROUNDING_SIZE = "grounding_size"
TOTAL_TIME = "total_time"

GRINGO = "gringo"
IDLV = "idlv"
NEWGROUND_GRINGO = "newground-gringo"
NEWGROUND_IDLV = "newground-idlv"

grounder_types = [GRINGO, IDLV, NEWGROUND_GRINGO, NEWGROUND_IDLV]


file_types = [GROUNDING_TIME, GROUNDING_SIZE, TOTAL_TIME]

S1 = "S1"

S2 = "S2"

S3_T4 = "S3-T4"
S3_T6 = "S3-T6"
S3_T8 = "S3-T8"

S4_T4 = "S4-T4"
S4_T6 = "S4-T6"
S4_T8 = "S4-T8"



In [491]:
problem_structure = {}

problem_structure[S1] = {}
problem_structure[S1]["directory"]= ["20230727"]
problem_structure[S1]["file_includes"] = "_sm_"

problem_structure[S2] = {}
problem_structure[S2]["directory"]= ["20230804","20230810"]
problem_structure[S2]["file_includes"] = "benchmark_output_nprc_"

problem_structure[S3_T4] = {}
problem_structure[S3_T4]["directory"]= ["20230803"]
problem_structure[S3_T4]["file_includes"] = "_output_tpg_star_4_"

problem_structure[S3_T6] = {}
problem_structure[S3_T6]["directory"]= ["20230803"]
problem_structure[S3_T6]["file_includes"] = "_output_tpg_star_6_"

problem_structure[S3_T8] = {}
problem_structure[S3_T8]["directory"]= ["20230803"]
problem_structure[S3_T8]["file_includes"] = "_output_tpg_star_8_"

problem_structure[S4_T4] = {}
problem_structure[S4_T4]["directory"]= ["20230805"]
problem_structure[S4_T4]["file_includes"] = "_output_tpg_02_star_4_"

problem_structure[S4_T6] = {}
problem_structure[S4_T6]["directory"]= ["20230805"]
problem_structure[S4_T6]["file_includes"] = "_output_tpg_02_star_6_"


problem_structure[S4_T8] = {}
problem_structure[S4_T8]["directory"]= ["20230805"]
problem_structure[S4_T8]["file_includes"] = "_output_tpg_02_star_8_"



In [492]:
type_structure = {}

for problem in problem_structure.keys():
    type_structure[problem] = {}
    type_structure[problem]["instance_size"] = 0
    for grounder_type in grounder_types:
        type_structure[problem][grounder_type] = {}
        for file_type in file_types:
            type_structure[problem][grounder_type][file_type] = {}
            type_structure[problem][grounder_type][file_type]["solved"] = 0
            
for problem in problem_structure.keys():
    type_structure[problem]["timeout"] = 1800


type_structure[S3_T4]["instance_size"] = 40 * 10
type_structure[S3_T6]["instance_size"] = 40 * 10
type_structure[S3_T8]["instance_size"] = 40 * 10


type_structure[S4_T4]["instance_size"] = 40 * 10
type_structure[S4_T6]["instance_size"] = 40 * 10
type_structure[S4_T8]["instance_size"] = 40 * 10

type_structure[S2]["timeout"] = 10800

#for problem in problems[-1:]:
#    type_structure[problem]["timeout"] = 10800
    

In [493]:
def read_csv_helper(directory, file, file_type):
    
    dens_structure = {}

    dens_structure["filename"] = file
    dens_structure["type"] = file_type
    file_data = pd.read_csv(os.path.join(directory, file), delimiter=',')    
    dens_structure["csv"] = file_data
    
    return (dens_structure)

In [494]:
def fill_type_structure(file_data, problem, type_structure):

    for column_name in list(file_data.columns):
        for grounder_type in grounder_types:
            if column_name.startswith(grounder_type) and "timeout-occurred" not in column_name and "timeout-occured" not in column_name:
                if "instance_size" not in type_structure[problem][grounder_type][file_type]:
                    type_structure[problem][grounder_type][file_type]["instance_size"] = []
                    
                if "size" in column_name:
                    new_column_name = GROUNDING_SIZE
                elif "grounding" in column_name:
                    new_column_name = GROUNDING_TIME
                else:
                    new_column_name = TOTAL_TIME
                    
                    
                if new_column_name not in type_structure[problem][grounder_type][file_type]:
                    type_structure[problem][grounder_type][file_type][new_column_name] = []


                list_instances = list(file_data["instance"])
                    

                instance_size_list = [int((instance_size_regex.search(instance)).group(0)) for instance in list_instances]


                type_structure[problem][grounder_type][file_type]["instance_size"] += instance_size_list
                
                timeout = type_structure[problem]["timeout"]

                raw_column_data = list(file_data[column_name])
                if "duration" in column_name:
                    parsed_column_data = [time if float(time) < timeout else -1 for time in raw_column_data]
                elif "size" in column_name:
                    parsed_column_data = [float(size)/1024 if float(size) <= 32*(math.pow(10,6)) else -1 for size in raw_column_data]
                else:
                    print("THIS SHOULD NEVER HAPPEN!")
                    
                solved = len([i for i in parsed_column_data if i >= 0])
                type_structure[problem][grounder_type][file_type]["solved"] += solved
                
                    
                type_structure[problem][grounder_type][file_type][new_column_name] += parsed_column_data
                



In [495]:
# Parse all files into type_structure
ds = sm

for problem in problem_structure.keys():
    ds = problem_structure[problem]
    
    for directory in ds["directory"]:
        for root, dirs, files in os.walk(directory):
            for file in files:

                if ds["file_includes"] in file and not "log" in file:

                    for file_type in file_types:
                        if file_type in file:
                            csv_dict = read_csv_helper(directory, file, file_type)



                            if "grounding_size" in file and not "S3" in problem and not "S4" in problem:
                                cur_size = type_structure[problem]["instance_size"]
                                instances = len(list(csv_dict["csv"]["instance"]))
                                type_structure[problem]["instance_size"] += instances

                            #density_structure[star][density][file_type] = local_density_structure

                            fill_type_structure(csv_dict["csv"], problem, type_structure)


In [496]:
type_structure

{'S1': {'instance_size': 50,
  'gringo': {'grounding_time': {'solved': 50,
    'instance_size': [1,
     2,
     3,
     4,
     5,
     6,
     7,
     8,
     9,
     10,
     11,
     12,
     13,
     14,
     15,
     16,
     17,
     18,
     19,
     20,
     21,
     22,
     23,
     24,
     25,
     26,
     27,
     28,
     29,
     30,
     31,
     32,
     33,
     34,
     35,
     36,
     37,
     38,
     39,
     40,
     41,
     42,
     43,
     44,
     45,
     46,
     47,
     48,
     49,
     50],
    'total_time': [256.34942269325256,
     254.80226230621335,
     254.4110403060913,
     249.20659804344177,
     263.7846868038177,
     268.3553800582886,
     261.9299201965332,
     256.53712725639343,
     263.0731408596039,
     255.02330994606015,
     260.1353154182434,
     263.7274031639099,
     255.48570919036865,
     257.8983931541443,
     254.02073168754575,
     258.0202317237854,
     263.82513666152954,
     258.0566484928131,
     275.681

In [497]:
#grounded_list = [["grounded", grounder_types[i]] for i in range(4)]
#solved_list = [["solved", grounder_types[i]] for i in range(4)]

grounded_list = [[] for i in range(4)]
solved_list = [[] for i in range(4)]
size_list = []

for problem in problem_structure.keys():
    counter = 0
    
    size_list.append(type_structure[problem]["instance_size"])
    
    for grounder in grounder_types:
        type_structure[problem][grounder]["total_solved"] = type_structure[problem][grounder][TOTAL_TIME]["solved"]
        
        solved_time = type_structure[problem][grounder][GROUNDING_TIME]["solved"]
        solved_size = type_structure[problem][grounder][GROUNDING_SIZE]["solved"]
        
        if solved_time < solved_size:
            type_structure[problem][grounder]["grounded"] = solved_time
        else:
            type_structure[problem][grounder]["grounded"] = solved_size
            
        solved_list[counter].append(type_structure[problem][grounder]["total_solved"])
        grounded_list[counter].append(type_structure[problem][grounder]["grounded"])
            
        counter += 1


        

In [498]:
instance_list = list(problem_structure.keys())

In [499]:
h1 = ["instances"] + ["grounded" for i in range(4)] + ["solved" for i in range(4)]
h2 = ["instances"] + [grounder_types[i] for i in range(4)] + [grounder_types[i] for i in range(4)]

columns = [h1,h2]

In [500]:
frame = pd.DataFrame(list(zip(instance_list, size_list, *grounded_list, *solved_list)))

In [502]:
frame

,0,1,2,3,4,5,6,7,8,9
0,S1,50,50,50,50,50,26,50,50,50
1,S2,270,47,205,45,223,27,93,27,100
2,S3-T4,400,132,141,216,218,116,137,214,215
3,S3-T6,400,76,80,200,199,65,74,198,197
4,S3-T8,400,53,53,177,176,49,50,175,170
5,S4-T4,400,50,79,122,122,48,77,122,122
6,S4-T6,400,20,32,68,67,19,32,68,66
7,S4-T8,400,14,20,56,58,13,20,56,57


In [503]:
frame.columns = ["instance"] + h2

In [504]:
frame

,instance,instances,gringo,idlv,newground-gringo,newground-idlv,gringo,idlv,newground-gringo,newground-idlv
0,S1,50,50,50,50,50,26,50,50,50
1,S2,270,47,205,45,223,27,93,27,100
2,S3-T4,400,132,141,216,218,116,137,214,215
3,S3-T6,400,76,80,200,199,65,74,198,197
4,S3-T8,400,53,53,177,176,49,50,175,170
5,S4-T4,400,50,79,122,122,48,77,122,122
6,S4-T6,400,20,32,68,67,19,32,68,66
7,S4-T8,400,14,20,56,58,13,20,56,57


In [505]:
frame = frame.set_index("instance")

In [506]:
frame

,instances,gringo,idlv,newground-gringo,newground-idlv,gringo,idlv,newground-gringo,newground-idlv
instance,,,,,,,,,
S1,50,50,50,50,50,26,50,50,50
S2,270,47,205,45,223,27,93,27,100
S3-T4,400,132,141,216,218,116,137,214,215
S3-T6,400,76,80,200,199,65,74,198,197
S3-T8,400,53,53,177,176,49,50,175,170
S4-T4,400,50,79,122,122,48,77,122,122
S4-T6,400,20,32,68,67,19,32,68,66
S4-T8,400,14,20,56,58,13,20,56,57


In [507]:
frame.columns = columns

In [508]:
frame

instances grounded                                      solved        
         instances   gringo idlv newground-gringo newground-idlv gringo idlv   
instance                                                                       
S1              50       50   50               50             50     26   50  \
S2             270       47  205               45            223     27   93   
S3-T4          400      132  141              216            218    116  137   
S3-T6          400       76   80              200            199     65   74   
S3-T8          400       53   53              177            176     49   50   
S4-T4          400       50   79              122            122     48   77   
S4-T6          400       20   32               68             67     19   32   
S4-T8          400       14   20               56             58     13   20   

                                          
         newground-gringo newground-idlv  
instance                                  
S1                     50             50  
S2                     27            100  
S3-T4                 214            215  
S3-T6                 198            197  
S3-T8                 175            170  
S4-T4                 122            122  
S4-T6                  68             66  
S4-T8                  56             57

In [509]:
my_latex = frame.to_latex()

In [510]:
print(my_latex)

\begin{tabular}{lrrrrrrrrr}
\toprule
 & instances & \multicolumn{4}{r}{grounded} & \multicolumn{4}{r}{solved} \\
 & instances & gringo & idlv & newground-gringo & newground-idlv & gringo & idlv & newground-gringo & newground-idlv \\
instance &  &  &  &  &  &  &  &  &  \\
\midrule
S1 & 50 & 50 & 50 & 50 & 50 & 26 & 50 & 50 & 50 \\
S2 & 270 & 47 & 205 & 45 & 223 & 27 & 93 & 27 & 100 \\
S3-T4 & 400 & 132 & 141 & 216 & 218 & 116 & 137 & 214 & 215 \\
S3-T6 & 400 & 76 & 80 & 200 & 199 & 65 & 74 & 198 & 197 \\
S3-T8 & 400 & 53 & 53 & 177 & 176 & 49 & 50 & 175 & 170 \\
S4-T4 & 400 & 50 & 79 & 122 & 122 & 48 & 77 & 122 & 122 \\
S4-T6 & 400 & 20 & 32 & 68 & 67 & 19 & 32 & 68 & 66 \\
S4-T8 & 400 & 14 & 20 & 56 & 58 & 13 & 20 & 56 & 57 \\
\bottomrule
\end{tabular}

